In [1]:
import math
import string
import time
from collections import defaultdict
import os
import sys
sys.path.append("/".join(os.getcwd().split("/")[:-2]))
from helpful_functions import *

In [2]:
with open("four_triangles_input.txt", "r") as f:
    prog_inp = f.readlines()

nr_groups = int(prog_inp.pop(0).rstrip())
print(f"{nr_groups} 4-triangle groups")
if prog_inp[-1] == '\n':
    del prog_inp[-1]
prog_inp = [[int(n) for n in line.rstrip().split(' ')] for line in prog_inp]

parsed_inp = []
current_group = []
for i, line in enumerate(prog_inp):
    current_group.append(line)
    if (i+1) % 4 == 0:
        parsed_inp.append(current_group)
        current_group = []
print(parsed_inp[:5])

4 4-triangle groups
[[[3, 4, 5], [5, 4, 3], [3, 4, 5], [4, 3, 5]], [[238, 185, 87], [109, 87, 28], [156, 109, 53], [303, 156, 153]], [[37, 20, 19], [80, 60, 100], [72, 80, 136], [37, 136, 164]], [[259, 210, 91], [259, 182, 105], [168, 117, 75], [30, 28, 26]]]


In [3]:
class Shape:
    """
    The angle (rad) between the i-th and the (i+1)-th sides is the i-th angle
    """

    def __init__(self, sides, angles=None):
        self.sides = sides
        self.angles = angles
        assert self.angles != None or len(self.sides) == 3, "angles are needed for a 4+ sided shape"

        if self.angles == None:
            self.angles = []
            for i in range(len(self.sides)):
                # Calculate angle using law of cosines
                num = self.sides[i]**2 + self.sides[(i+1)%3]**2 - self.sides[(i+2)%3]**2
                denom = 2 * self.sides[i] * self.sides[(i+1)%3]
                self.angles.append(math.acos(num/denom))
    
    def __eq__(self, other):
        if not isinstance(other, Shape): return False

        if len(self.sides) != len(other.sides): return False
        for i in range(len(self.sides)):
            eq = True
            for j in range(len(self.sides)):
                if self.sides[(j+i) % len(self.sides)] != other.sides[j]:
                    eq = False
                    break
            if eq: return True
            req = True
            for j in range(len(self.sides)):
                if self.sides[(j+i) % len(self.sides)] != other.sides[-j]:
                    req = False
                    break
            if req: return True
                
        return False
    
    def add_tri(self, other, force=False):
        """
        sticks the first side of `other` onto the last side of `self`
        """

        assert len(other.sides) == 3, "other must be a 3-sided triangle"
        assert self.sides[-1] == other.sides[0] or force, "cannot combine shapes with mismatched side lengths"

        # Make the new shape
        self.sides[-1] = other.sides[1]
        self.sides.append(other.sides[2])
        # Recalculate angles
        last_angle = self.angles[-1]
        self.angles[-2] += other.angles[0]
        self.angles[-1] = other.angles[1]
        self.angles.append(last_angle + other.angles[2])

        return self
    
    def copy(self):
        return Shape(self.sides[:], self.angles[:])
    
    def __repr__(self):
        return f"Shape({str(self.sides)}, {str(self.angles)})"

    def __str__(self):
        return self.__repr__()

    def flip(self):
        self.sides = self.sides[::-1]
        self.angles = self.angles[::-1]
        self.angles = self.angles[1:] + self.angles[:1]
        return self
    
    def flipped(self):
        return self.copy().flip()
    
    def shift(self, n=1):
        self.sides = self.sides[-n:] + self.sides[:-n]
        self.angles = self.angles[-n:] + self.angles[:-n]
        return self
    
    def shifted(self, n=1):
        return self.copy().shift(n)
    
    def simplify(self):
        org_nr_sides = len(self.sides)
        skip_next = False
        for i in range(org_nr_sides-1, -1, -1):
            if skip_next:
                skip_next = False
                continue

            # Straight line (two collinear edges)
            if abs(self.angles[i] - math.pi) < 0.01:
                self.sides[(i+1)%org_nr_sides] += self.sides[i]
                del self.sides[i]
                del self.angles[i]

            # Edge double backs on itself (two edges overlapping)
            elif abs(self.angles[i] - 2*math.pi) < 0.01:
                len_diff = self.sides[(i+1)%org_nr_sides] - self.sides[i]

                if len_diff == 0:
                    self.angles[i-1] += self.angles[(i+1)%org_nr_sides]
                    if i == org_nr_sides-1:
                        del self.sides[i]
                        del self.angles[i]
                        # i+1 will loop around to the beginning
                        del self.sides[0]
                        del self.angles[0]
                        # deleting something from the beginning so the next iteration needs to be skipped
                        skip_next = True
                    else:
                        del self.sides[i:i+2]
                        del self.angles[i:i+2]

                elif len_diff <= 0:
                    self.sides[(i+1)%org_nr_sides] = abs(len_diff)
                    self.angles[(i+1)%org_nr_sides] += math.pi
                    del self.sides[i]
                    del self.angles[i]

                elif len_diff >= 0:
                    self.sides[(i+1)%org_nr_sides] = len_diff
                    self.angles[i-1] += math.pi
                    del self.sides[i]
                    del self.angles[i]
        return self

In [4]:
def combine_to_triangle(group, prev_shape=None, depth=0):
    depth += 1
    # The depth is how many triangles are grouped together in prev_shape

    if depth == 1:
        prev_shape = Shape(group[0])
    elif depth == 4:
        if len(prev_shape.sides) == 3: return prev_shape
        return False

    triangle = Shape(group[depth])
    flipped_triangle = triangle.flipped()
    
    # For each triangle side
    for t_i in range(3):
        # Find all sides in prev_shape the same length
        ps_indices = find_indices(prev_shape.sides, triangle.sides[t_i])
        # For each side with the same length, try adding the triangle & flipped triangle to that side
        for ps_i in ps_indices:
            # Shift the prev_shape and triangle so that the matching sides are the last and first sides respectively (necessary for add_tri to work)
            new_shape_1 = prev_shape.shifted(-ps_i - 1)
            new_shape_2 = new_shape_1.copy()

            shifted_tri = triangle.shifted(-t_i)
            shifted_flipped_tri = flipped_triangle.shifted(t_i - 2)

            new_shape_1.add_tri(shifted_tri).simplify()
            new_shape_2.add_tri(shifted_flipped_tri).simplify()
            if (res := combine_to_triangle(group, new_shape_1, depth)) != False:
                return res
            elif (res := combine_to_triangle(group, new_shape_2, depth)) != False:
                return res

    # At this point a triangle could not be formed by just sticking sides of equal length together
    # This will try sticking two triangles onto one side
    if depth == 1:
        for ps_i, side in enumerate(prev_shape.sides):
            # Get every possible pair of triangles
            for t1_i in range(depth, len(group)-1):
                for t2_i in range(depth+1, len(group)):
                    t1 = Shape(group[t1_i])
                    t2 = Shape(group[t2_i])
                    # Every side pair for every triangle pair
                    for s1_i in range(3):
                        for s2_i in range(3):
                            # Check if the side pair adds up to the side in prev_shape
                            if t1.sides[s1_i] + t2.sides[s2_i] == side:
                                # Find the one orientation for the triangles such that 2 of the new triange's sides are collinear
                                #  /||\  |  /| /| |\ |\  |\  /|
                                # /_||_\ | /_|/_| |_\|_\ |_\/_|
                                # `-..-` | `-..-` `-..-` `-..-`
                                #  This  |  not These
                                # (bottom is prev_shape)

                                # If one triangle is an isosceles triangle there could be 2 possible orientations, but the second orientaiton is equal to the first so who cares about it
                                about_equals_pi = lambda a: abs(a - math.pi) <= 0.01
                                if about_equals_pi(t1.angles[s1_i-1] + t2.angles[s2_i]):
                                    flip = (False, False)
                                elif about_equals_pi(t1.angles[s1_i-1] + t2.angles[s2_i-1]):
                                    flip = (False, True)
                                elif about_equals_pi(t1.angles[s1_i] + t2.angles[s2_i]):
                                    flip = (True, False)
                                elif about_equals_pi(t1.angles[s1_i] + t2.angles[s2_i-1]):
                                    flip = (True, True)
                                else:
                                    continue # No valid orientation

                                t1_m = t1.copy()
                                t2_m = t2.copy()
                                # Switching the order and flipping both triangles will also be a valid way to stick the two triangles onto prev_shape
                                t1_mf = t1.copy()
                                t2_mf = t2.copy()

                                if flip[0]:
                                    t1_m.flip().shift(s1_i - 2)
                                    t1_mf.shift(-s1_i)
                                else:
                                    t1_m.shift(-s1_i)
                                    t1_mf.flip().shift(s1_i - 2)

                                if flip[1]:
                                    t2_m.flip().shift(s2_i - 2)
                                    t2_mf.shift(-s2_i)
                                else:
                                    t2_m.shift(-s2_i)
                                    t2_mf.flip().shift(s2_i - 2)

                                new_shape_1 = prev_shape.shifted(-ps_i - 1) # for *_m
                                new_shape_2 = new_shape_1.copy() # for *_mf

                                last_angle = new_shape_1.angles[-1]
                                # Add the first triangle
                                new_shape_1.add_tri(t1_m, force=True)
                                # This messed up the last edge (it is no longer a closed polygon) so now we have to clean it up
                                # Add a new edge closing the shape
                                new_shape_1.sides.append(t2_m.sides[0])
                                # Clean up the angles
                                new_shape_1.angles[-1] = t1_m.angles[-1] + math.pi
                                new_shape_1.angles.append(last_angle)
                                # Finally we can add the second triangle
                                new_shape_1.add_tri(t2_m).simplify()


                                last_angle = new_shape_2.angles[-1]
                                # Add the first triangle
                                new_shape_2.add_tri(t2_mf, force=True)
                                # This messed up the last edge (it is no longer a closed polygon) so now we have to clean it up
                                # Add a new edge closing the shape
                                new_shape_2.sides.append(t1_mf.sides[0])
                                # Clean up the angles
                                new_shape_2.angles[-1] = t2_mf.angles[-1] + math.pi
                                new_shape_2.angles.append(last_angle)
                                # Finally we can add the second triangle
                                new_shape_2.add_tri(t1_mf).simplify()

                                
                                last_triangle_i = [False] + [True]*3
                                last_triangle_i[t1_i] = False
                                last_triangle_i[t2_i] = False
                                last_triangle_i = last_triangle_i.index(True)
                                new_group = [None, None, None, group[last_triangle_i]]

                                if (res := combine_to_triangle(new_group, new_shape_1, 2)) != False:
                                    return res
                                if (res := combine_to_triangle(new_group, new_shape_2, 2)) != False:
                                    return res

    return False

results = []
for i in range(4):
    results.append(combine_to_triangle(parsed_inp[i]))
    # An unfortunate first triangle can sometimes be impossible to solve using my method
    # However, starting with any other triangle is doable so just run the algorithm again with a different triangle order
    if results[-1] == False:
        results[-1] == combine_to_triangle(parsed_inp[i][::-1])
    print(f"test case {i}:", results[-1])

with open("four_triangles_output.txt", 'w') as f:
    f.write("\n".join(["No" if r==False else "Yes" for r in results]))

test case 0: Shape([6, 8, 10], [1.5707963267948966, 0.6435011087932843, 0.9272952180016123])
test case 1: Shape([472, 185, 303], [0.3302973548292537, 2.611957993778215, 0.19933730498232413])
test case 2: False
test case 3: Shape([259, 259, 168], [0.6605947096585074, 1.2404989719656427, 1.2404989719656427])


In [5]:
import turtle as turt

In [6]:
def draw_shape(shape, scale=1):
    turt.reset()
    turt.st()
    for i in range(len(shape.sides)):
        turt.forward(shape.sides[i]*scale)
        turt.left(180 - shape.angles[i]*360/(2*math.pi))
    # turt.ht()

In [7]:
draw_shape(results[3], 1)

In [8]:
turt.bye()